In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import warnings
import missingno as msno
import seaborn as sns 
import matplotlib.pyplot as plt

warnings.filterwarnings("ignore")


In [ ]:
train = pd.read_csv("../input/pubg-finish-placement-prediction/train_V2.csv")


In [ ]:
train.head()

In [ ]:
train.info()

* Id - 플레이어의 Id
* groupId - 경기 내의 그룹을 식별하는 ID. 현재 그룹의 선수들이 서로 다른 경기에서 경기한다면, 그들은 매번 다른 groupId를 갖게 될 것이다.
* matchId - 경기를 식별하기 위한 ID. train set과 test set에 모두 있는 시합은 없다.
* matchDuration - 경기 시간.
* matchType - 경기 종류(duo, quad ...).
* assists - 팀 동료들과 같이 죽인 적 수이다.
* boosts - 부스트 아이템 사용한 수.
* damageDealt - 가한 데미지 총량. Note: 자신에게 가한 데미지는 제외.
* DBNOs - 빈사상태로 만든 적의 수.
* headshotKills - 헤드샷 으로 처리한 적의 수.
* heals - 치료 아이템 사용 수.
* killPlace - 경기에서 처치한 적의 수 랭킹.
* killPoints - 플레이어의 처치 기반 외부 랭킹 (Elo 방식의 순위). rankPoints에서 -1이 아닌 값이 있는 경우, killPoints에서 0은 "없음"으로 처리되어야 한다.
* kills - 처치한 적의 수.
* killStreaks - 단기간에 가장 많이 처치한 적의 최대치.
* longestKill - 플레이어가 적을 죽인 가장 긴 거리. 선수를 다운시키고 멀리 운전하는 것이 가장 긴 처치로 이어질 수 있기 때문에 오해의 소지가 있을 수 있다.
* maxPlace - 경기에서 가장 순위가 낮은 것에 대한 데이터. 이것은 순위를 건너뛸 수도 있기 때문에 numGroups와 일치하지 않을 수 있다.
* numGroups - 경기에 있는 팀의 수.
* rankPoints - Elo 방식의 플레이어 랭킹. 다음 버전의 API에서는 삭제될 예정이기 때문에 사용에 주의. '-1' 값은 순위가 "None" 이다.
* revives - 플레이어가 팀원 회복 시킨 수.
* rideDistance - 차량으로 이동한 거리(단위 : 미터).
* roadKills - 차량으로 죽인 플레이어 수.
* swimDistance - 수영한 거리(단위 : 미터).
* teamKills - 팀킬한 횟수.
* vehicleDestroys - 차량을 폭파시킨 횟수.
* walkDistance - 걸은 총 거리(단위 : 미터).
* weaponsAcquired - 무기 얻은 갯수.
* winPoints - 플레이어의 승리 기반 외부 랭킹 (Elo 방식의 순위). rankPoints에서 -1이 아닌 값이 있는 경우, winPoints에서 0은 "없음"으로 처리되어야 한다.
* winPlacePerc - 예측 목표. 순위의 퍼센트로 표시되며, 1이면 경기에서 1등이고 0이면 경기에서 꼴지 했다는 것이다. 이것은 numGroups로 계산되는게 아니라, maxPlace로 계산되기 때문에 누락되는 것이 있을수도 있다.

In [ ]:
print(train.isnull().sum())#결측치 확인

In [ ]:
train[train['winPlacePerc'].isna() == True]

In [ ]:
train =train.dropna()

In [ ]:
train['matchType']

In [ ]:
print(train.isnull().sum())#결측치 확인

In [ ]:
train

# matchType 분류

In [ ]:
f,ax = plt.subplots(figsize=(15,8))
sns.barplot(train.matchType.value_counts().values,train.matchType.value_counts().index)
plt.xlabel('Count')
plt.ylabel('matchType')
plt.show()
train['matchType'].value_counts()

In [ ]:
mapper = lambda x: 'solo' if ('solo' in x) else 'duo' if ('duo' in x) else 'squad'


In [ ]:
train['matchType']

In [ ]:
train['matchType'] = train['matchType'].apply(mapper)
sns.countplot(train['matchType'])
plt.title('Count of different types of match')
plt.show()

In [ ]:
train['matchType']

# 상관관계

In [ ]:
f,ax = plt.subplots(figsize=(15,15))
ax = sns.heatmap(train.corr(),annot=True,fmt= '.1f',linewidths=.5)

walkDistance, kill, weaponsAcquired, heal&boosts 시각화

# KILL

In [ ]:
data = train.copy()
data.loc[data['kills'] > data['kills'].quantile(0.99)] = '8+'
plt.figure(figsize=(15,8))
sns.countplot(data['kills'].astype(str).sort_values())
plt.title("Kill Count")
plt.show()

In [ ]:

kills = train.copy()

kills['killsCategories'] = pd.cut(kills['kills'], [-1, 0, 2, 5, 10, 60], labels=[
    '0_kills','1-2_kills', '3-5_kills', '6-10_kills', '10+_kills'])

plt.figure(figsize=(15,8))
sns.boxplot(x="killsCategories", y="winPlacePerc", data=kills)
plt.show()

# walkDistance

In [ ]:
sns.jointplot(data = train, x = "winPlacePerc", y = "walkDistance", height = 10,  ratio=3)


# weaponsAcquired

In [ ]:
sns.jointplot(data = train, x = "winPlacePerc", y = "weaponsAcquired", height = 10,  ratio=3)


# heal & boosts

In [ ]:
sns.jointplot(data = train, x = "winPlacePerc", y = "heals", height = 10, ratio=3)
sns.jointplot(data = train, x = "winPlacePerc", y = "boosts", height = 10, ratio=3)


In [ ]:
sns.set()
cols = ['winPlacePerc', 'walkDistance', 'boosts','heals', 'weaponsAcquired', 'killPlace']
sns.pairplot(train[cols], size = 2.5)
plt.show()